In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Especifica la ruta al archivo CSV
path = r'C:\\Users\\joset\\OneDrive\\Documentos\\Master Big Data UCM\\18-TFN\NaturgyProject\\data'
# Lee el archivo CSV en un DataFrame de pandas
clientes_y_consumo_original = pd.read_csv(path + '\\Originales\\clientes_y_consumo.csv', encoding = 'latin1')
Coordenadas_CCPP_original = pd.read_csv(path +'\\Originales\\Coordenadas_CCPP.csv', encoding = 'latin1')
AEMET_Realidad_clean_original = pd.read_csv(path +'\\Cleaning\\AEMET_Realidad_clean.csv', encoding = 'latin1')
provincias_temperaturas_promedio_original = pd.read_csv(path + '\\Cleaning\\provincias_temperaturas_promedio.csv', encoding = 'latin1')

C:\Users\joset\AppData\Local\Temp\ipykernel_10296\3451541042.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  clientes_y_consumo_original = pd.read_csv(path + '\\Originales\\clientes_y_consumo.csv', encoding = 'latin1')


In [3]:
clientes_y_consumo = clientes_y_consumo_original.copy()
Coordenadas_CCPP = Coordenadas_CCPP_original.copy()
AEMET_Realidad_clean = AEMET_Realidad_clean_original.copy()
provincias_temperaturas_promedio = provincias_temperaturas_promedio_original.copy()

In [4]:
clientes_y_consumo.head(3)

,MES,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura
0,01/01/2019,NaN,01001,GNS,3.2,1,2314.475921,0
1,01/01/2019,NaN,01003,GNS,3.1,1,2167.612737,0
2,01/01/2019,NaN,01006,GNS,3.2,1,1157.655549,0


In [5]:
# Reemplazar cualquier valor no numérico con NaN
clientes_y_consumo['COD_POST'] = pd.to_numeric(clientes_y_consumo['COD_POST'], errors='coerce')
# Llenar valores NaN con 0 y convertir la columna a enteros
clientes_y_consumo['COD_POST'] = clientes_y_consumo['COD_POST'].fillna(0).astype(int)
clientes_y_consumo['FECHA'] = pd.to_datetime(clientes_y_consumo['MES'], format='%d/%m/%Y')
clientes_y_consumo = clientes_y_consumo.drop(columns=['MES'])
clientes_y_consumo['Comercializadora'] = clientes_y_consumo['Comercializadora'].replace('**', 'NC')
clientes_y_consumo['comer'] = clientes_y_consumo['Comercializadora'].map({'GNS': '33','SUR': '52', 'NC': '762'})
# Nos quedamos unicamente con el tipo de tarifas que nos interesan
tarifas = ['RL.2', 'RL.1', 'RL.3']
clientes_y_consumo = clientes_y_consumo[clientes_y_consumo['Tarifa_ATR'].isin(tarifas)]
clientes_y_consumo.head(3)

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer
621622,NaN,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33
621623,NaN,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33
621624,NaN,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33


In [6]:
Coordenadas_CCPP.head(3)

,codigo_postal,latitud,longitud
0,52080,35.3167,-2.95
1,52071,35.3167,-2.95
2,52070,35.3167,-2.95


In [7]:
#hacemos un merge para incorporar las latitudes y longitudes a partir de los Codigos postales
clientes_y_consumo_mejorado = clientes_y_consumo.copy()
# Realizar la combinación
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.merge(
    Coordenadas_CCPP[['codigo_postal', 'latitud', 'longitud']],
    left_on='COD_POST',
    right_on='codigo_postal',
    how='left'
)
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.rename(columns={'latitud': 'LATITUD', 'longitud': 'LONGITUD'})
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.drop(columns=['codigo_postal'])
clientes_y_consumo_mejorado['GOOGLE_MAPS'] = clientes_y_consumo_mejorado.apply(lambda row: f"{row['LATITUD']},{row['LONGITUD']}" if pd.notnull(row['LATITUD']) and pd.notnull(row['LONGITUD']) else None, axis=1)
clientes_y_consumo_mejorado.head(3)

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer,LATITUD,LONGITUD,GOOGLE_MAPS
0,NaN,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667"
1,NaN,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667"
2,NaN,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667"


In [8]:
#creamos un diccionario a partir de las coordenada para colocar la provincia correspondientes:
provincias_coords = {
    'TARRAGONA': {'LATITUD': 41.1189, 'LONGITUD': 1.2453},
    'BARCELONA': {'LATITUD': 41.3851, 'LONGITUD': 2.1734},
    'GIRONA': {'LATITUD': 41.9794, 'LONGITUD': 2.8214},
    'GIPUZKOA': {'LATITUD': 43.3128, 'LONGITUD': -1.9812},
    'BIZKAIA': {'LATITUD': 43.2630, 'LONGITUD': -2.934},
    'CANTABRIA': {'LATITUD': 43.1828, 'LONGITUD': -3.9878},
    'ASTURIAS': {'LATITUD': 43.3603, 'LONGITUD': -5.8448},
    'A CORUÑA': {'LATITUD': 43.3623, 'LONGITUD': -8.4115},
    'PONTEVEDRA': {'LATITUD': 42.4299, 'LONGITUD': -8.6444},
    'LUGO': {'LATITUD': 43.0125, 'LONGITUD': -7.5558},
    'LEON': {'LATITUD': 42.5987, 'LONGITUD': -5.5671},
    'OURENSE': {'LATITUD': 42.336, 'LONGITUD': -7.8639},
    'SORIA': {'LATITUD': 41.7636, 'LONGITUD': -2.4646},
    'BURGOS': {'LATITUD': 42.3439, 'LONGITUD': -3.6969},
    'PALENCIA': {'LATITUD': 42.0095, 'LONGITUD': -4.5283},
    'VALLADOLID': {'LATITUD': 41.6529, 'LONGITUD': -4.7244},
    'AVILA': {'LATITUD': 40.6578, 'LONGITUD': -4.699},
    'SEGOVIA': {'LATITUD': 40.9429, 'LONGITUD': -4.1088},
    'ZAMORA': {'LATITUD': 41.5035, 'LONGITUD': -5.7432},
    'SALAMANCA': {'LATITUD': 40.9701, 'LONGITUD': -5.6635},
    'MADRID': {'LATITUD': 40.4168, 'LONGITUD': -3.7038},
    'GUADALAJARA': {'LATITUD': 40.633, 'LONGITUD': -3.1669},
    'TOLEDO': {'LATITUD': 39.8628, 'LONGITUD': -4.0273},
    'CACERES': {'LATITUD': 39.4765, 'LONGITUD': -6.3722},
    'CIUDAD REAL': {'LATITUD': 38.9867, 'LONGITUD': -3.9272},
    'BADAJOZ': {'LATITUD': 38.8786, 'LONGITUD': -6.9705},
    'HUELVA': {'LATITUD': 37.2614, 'LONGITUD': -6.9447},
    'JAEN': {'LATITUD': 37.7796, 'LONGITUD': -3.7849},
    'CORDOBA': {'LATITUD': 37.8882, 'LONGITUD': -4.7794},
    'GRANADA': {'LATITUD': 37.1773, 'LONGITUD': -3.5986},
    'SEVILLA': {'LATITUD': 37.3886, 'LONGITUD': -5.9823},
    'CADIZ': {'LATITUD': 36.516, 'LONGITUD': -6.2994},
    'MALAGA': {'LATITUD': 36.7213, 'LONGITUD': -4.4214},
    'ALMERIA': {'LATITUD': 36.834, 'LONGITUD': -2.4637},
    'MURCIA': {'LATITUD': 37.9834, 'LONGITUD': -1.1282},
    'ALICANTE': {'LATITUD': 38.3452, 'LONGITUD': -0.481},
    'CUENCA': {'LATITUD': 40.0704, 'LONGITUD': -2.1374},
    'ALBACETE': {'LATITUD': 38.995, 'LONGITUD': -1.8564},
    'TERUEL': {'LATITUD': 40.344, 'LONGITUD': -1.1069},
    'VALENCIA': {'LATITUD': 39.4699, 'LONGITUD': -0.3763},
    'CASTELLON': {'LATITUD': 39.9861, 'LONGITUD': -0.0363},
    'ARABA/ALAVA': {'LATITUD': 42.8461, 'LONGITUD': -2.6716},
    'LA RIOJA': {'LATITUD': 42.2871, 'LONGITUD': -2.5396},
    'NAVARRA': {'LATITUD': 42.6954, 'LONGITUD': -1.6761},
    'ZARAGOZA': {'LATITUD': 41.6488, 'LONGITUD': -0.8891},
    'LLEIDA': {'LATITUD': 41.6167, 'LONGITUD': 0.6222},
    'HUESCA': {'LATITUD': 42.1401, 'LONGITUD': -0.4087},
    'BALEARES': {'LATITUD': 39.5712, 'LONGITUD': 2.6466},
    'LAS PALMAS': {'LATITUD': 28.1235, 'LONGITUD': -15.4363}
}

In [9]:
#creamos una funcion para revisar diccionario y colocar la provincia referente a esa latidud y longitud:
from scipy.spatial import distance
df_provincias = pd.DataFrame(provincias_coords).T.reset_index().rename(columns={'index': 'PROVINCIA'})

def encontrar_provincia(latitud, longitud):
    distancias = df_provincias.apply(
        lambda row: distance.euclidean((latitud, longitud), (row['LATITUD'], row['LONGITUD'])),
        axis=1
    )
    idx_min_dist = distancias.idxmin()
    return df_provincias.loc[idx_min_dist, 'PROVINCIA']

def agregar_provincia(df):
    df=df.copy()
    def get_provincia(row):
        if pd.notnull(row['LATITUD']) and pd.notnull(row['LONGITUD']):
            return encontrar_provincia(row['LATITUD'], row['LONGITUD'])
        else:
            return np.nan
    
    df['PROVINCIA'] = df.apply(get_provincia, axis=1)
    return df

In [10]:
#Colocamos las provincias (Esta linea demora aproximadamente 21m no ejecutar si no lo necesitamos):
clientes_y_consumo_mejorado = agregar_provincia(clientes_y_consumo_mejorado)
clientes_y_consumo_mejorado.head(3)

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer,LATITUD,LONGITUD,GOOGLE_MAPS,PROVINCIA
0,NaN,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA
1,NaN,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA
2,NaN,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA


In [11]:
#Armamos un pequeño dataset a partir de nuestro AEMET_Realidad_clean
Lista_AEMET_Log_Lat = AEMET_Realidad_clean[['NOMBRE', 'LONGITUD', 'LATITUD']].drop_duplicates().reset_index(drop=True)
Lista_AEMET_Log_Lat.head(3)

,NOMBRE,LONGITUD,LATITUD
0,REUS/AEROPUERTO,1.09491,41.0842
1,"TARRAGONA, FAC. GEOGRAFÃA",1.14571,41.0726
2,BARCELONA/AEROPUERTO,2.04121,41.1734


In [12]:
# Función para calcular la distancia euclídea y encontrar el `NOMBRE` más cercano
def encontrar_nombre(latitud, longitud):
    if pd.isnull(latitud) or pd.isnull(longitud):
        return np.nan
    # Filtrar filas con valores válidos en Lista_AEMET_Log_Lat
    valid_coords = Lista_AEMET_Log_Lat.dropna(subset=['LATITUD', 'LONGITUD'])
    distancias = valid_coords.apply(
        lambda row: distance.euclidean((latitud, longitud), (row['LATITUD'], row['LONGITUD'])),
        axis=1
    )
    idx_min_dist = distancias.idxmin()
    return valid_coords.loc[idx_min_dist, 'NOMBRE']

# Aplicar la función para agregar la columna 'NOMBRE' al DataFrame clientes_y_consumo_mejorado
def agregar_nombre(df):
    df=df.copy()
    df['NOMBRE'] = df.apply(lambda row: encontrar_nombre(row['LATITUD'], row['LONGITUD']), axis=1)
    return df

In [13]:
# Colocamos la estacion mas proxima, este mapeo demora aproximadamente 48min no ejecutar si no lo necesitamos
clientes_y_consumo_mejorado = agregar_nombre(clientes_y_consumo_mejorado)

In [14]:
#Vamos a ver como quedo nuestro dataset
clientes_y_consumo_mejorado.head(3)

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer,LATITUD,LONGITUD,GOOGLE_MAPS,PROVINCIA,NOMBRE
0,NaN,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO
1,NaN,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO
2,NaN,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO


In [15]:
# Realizar el merge basado en 'ESTACION_CERCANA' y 'FECHA'
# Asegúrate de que ambas columnas 'FECHA' en los DataFrames estén en formato datetime
clientes_y_consumo_mejorado['FECHA'] = pd.to_datetime(clientes_y_consumo_mejorado['FECHA'], errors='coerce')
AEMET_Realidad_clean['FECHA'] = pd.to_datetime(AEMET_Realidad_clean['FECHA'], errors='coerce')
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.merge(
    AEMET_Realidad_clean[['NOMBRE', 'FECHA', 'MAX', 'MIN']],
    on=['NOMBRE', 'FECHA'],
    how='left')
clientes_y_consumo_mejorado['MAX'].fillna('0', inplace=True)
clientes_y_consumo_mejorado['MIN'].fillna('0', inplace=True)
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.rename(columns={'NOMBRE': 'ESTACION_CERCANA'})
clientes_y_consumo_mejorado.head(3)

C:\Users\joset\AppData\Local\Temp\ipykernel_10296\1056801690.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clientes_y_consumo_mejorado['MAX'].fillna('0', inplace=True)
C:\Users\joset\AppData\Local\Temp\ipykernel_10296\1056801690.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  clientes_y_consumo_mejorado['MAX'].fillna('0', inplace=True)
C:\User

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer,LATITUD,LONGITUD,GOOGLE_MAPS,PROVINCIA,ESTACION_CERCANA,MAX,MIN
0,NaN,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6
1,NaN,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6
2,NaN,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6


In [16]:
clientes_y_consumo_mejorado.to_csv(path + '\\Cleaning\\clientes_y_consumo_mejorado.csv', index=False)

In [17]:
#armamos un pequeño dataset con las provincias y sus estaciones
# Crear la lista de estaciones y provincias eliminando duplicados, NaN y reseteando el índice
Lista_Estacion_Provincia = clientes_y_consumo_mejorado[['COD_POST', 'ESTACION_CERCANA', 'PROVINCIA', 'GOOGLE_MAPS']].drop_duplicates().dropna().reset_index(drop=True)
# Mostrar el resultado
Lista_Estacion_Provincia.head(3)


,COD_POST,ESTACION_CERCANA,PROVINCIA,GOOGLE_MAPS
0,1001,BILBAO AEROPUERTO,ARABA/ALAVA,"42.85,-2.6667"
1,1002,BILBAO AEROPUERTO,ARABA/ALAVA,"42.85,-2.6667"
2,1003,BILBAO AEROPUERTO,ARABA/ALAVA,"42.85,-2.6667"


In [18]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
Lista_Estacion_Provincia.to_csv(path + '\\Cleaning\\Lista_Estacion_Provincia.csv', index=False)

In [19]:
#creamor un pequeño dastaset que contenga CODIGOS POSTALES, COMERCIALIZADORA, CODIGOS DE COMERCIALIZADOS Y PCTD
Lista_comer_cp__pctd = clientes_y_consumo_mejorado[['PCTD', 'COD_POST', 'Comercializadora', 'comer']]
Lista_comer_cp__pctd = Lista_comer_cp__pctd.dropna()
Lista_comer_cp__pctd = Lista_comer_cp__pctd.drop_duplicates()
Lista_comer_cp__pctd.head(3)

,PCTD,COD_POST,Comercializadora,comer
4248,03A,15320,GNS,33
4251,03A,15320,SUR,52
4254,03A,15403,GNS,33


In [20]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
Lista_comer_cp__pctd.to_csv(path + '\\Cleaning\\Lista_comer_cp__pctd.csv', index=False)

In [21]:
# Función para completar valores faltantes en la columna 'PCTD'
def completar_pctd(row, ref_dataset):
    if pd.isna(row['PCTD']) or row['PCTD'] == '' or row['PCTD'] == '0':
        ref_row = ref_dataset[(ref_dataset['COD_POST'] == row['COD_POST']) & 
                              (ref_dataset['Comercializadora'] == row['Comercializadora']) & 
                              (ref_dataset['comer'] == row['comer'])]
        if not ref_row.empty:
            row['PCTD'] = ref_row.iloc[0]['PCTD']
    return row

In [22]:
# Aplicar la función para completar la columna 'PCTD' en el dataset completo (9mins)
clientes_y_consumo_mejorado = clientes_y_consumo_mejorado.apply(completar_pctd, axis=1, ref_dataset=Lista_comer_cp__pctd)
clientes_y_consumo_mejorado.head(3)

,PCTD,COD_POST,Comercializadora,Tarifa_ATR,nCUPS,Consumo_kWh,n_fallo_lectura,FECHA,comer,LATITUD,LONGITUD,GOOGLE_MAPS,PROVINCIA,ESTACION_CERCANA,MAX,MIN
0,38,1001,GNS,RL.2,1,249.418774,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6
1,38,1002,GNS,RL.2,1,53.000000,0,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6
2,38,1003,GNS,RL.2,2,937.474177,1,2021-10-01,33,42.85,-2.6667,"42.85,-2.6667",ARABA/ALAVA,BILBAO AEROPUERTO,23.3,10.6


In [23]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
clientes_y_consumo_mejorado.to_csv(path + '\\Cleaning\\clientes_y_consumo_mejorado.csv', index=False)

In [24]:
def sumar_consumo_por_provincia_y_mes(df):
    """
    Esta función agrupa el DataFrame por 'Provincia' y por mes, y suma los valores de 'Consumo_kWh'.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene los datos.
    
    Retorna:
    pandas.DataFrame: Un DataFrame con los valores de 'Consumo_kWh' sumados por 'Provincia' y por mes.
    """
    # Asegúrate de que las fechas estén en el tipo correcto
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    
    # Crear una nueva columna para el mes y el año
    df['AÑO_MES'] = df['FECHA'].dt.to_period('M')
        
    # Agrupar por 'Provincia' y 'AÑO_MES' y sumar 'Consumo_kWh'
    df_suma = df.groupby(['PROVINCIA', 'AÑO_MES'])['Consumo_kWh'].sum().reset_index()
    # fomrato mas legible de la columna consumo_kwh
    df_suma['Consumo_kWh'] = df_suma['Consumo_kWh'].map('{:,.2f}'.format)
    
    return df_suma

In [25]:
consumo_provincia_mes = sumar_consumo_por_provincia_y_mes(clientes_y_consumo_mejorado)
consumo_provincia_mes['Consumo_kWh'] = consumo_provincia_mes['Consumo_kWh'].str.replace(',', '').astype(float)
consumo_provincia_mes.head(3)

,PROVINCIA,AÑO_MES,Consumo_kWh
0,A CORUÑA,2021-10,10695536.73
1,A CORUÑA,2021-11,24993254.09
2,A CORUÑA,2021-12,31367381.77


In [26]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
consumo_provincia_mes.to_csv(path + '\\Cleaning\\consumo_provincia_mes.csv', index=False)

In [27]:
def calcular_porcentaje_por_provincia(df):
    """
    Esta función calcula el porcentaje de consumo por provincia respecto al total del consumo del mes.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene las columnas 'PROVINCIA', 'AÑO_MES' y 'Consumo_kWh'.
    
    Retorna:
    pandas.DataFrame: Un DataFrame con una columna adicional '%_Consumo' que representa el porcentaje de consumo
                      de cada provincia respecto al total del mes.
    """
    # Agrupar por 'AÑO_MES' y calcular el total de 'Consumo_kWh' para cada mes
    total_por_mes = df.groupby('AÑO_MES')['Consumo_kWh'].sum().reset_index()
    total_por_mes = total_por_mes.rename(columns={'Consumo_kWh': 'Total_Consumo_kWh'})
    
    # Unir el total por mes con el DataFrame original
    df = pd.merge(df, total_por_mes, on='AÑO_MES')
    
    # Calcular el porcentaje de consumo por provincia respecto al total del mes
    df['%_Consumo'] = ((df['Consumo_kWh'] / df['Total_Consumo_kWh']) * 100).round(2)
    
    # Eliminar la columna 'Total_Consumo_kWh' ya que no es necesaria en el resultado final
    df = df.drop(columns=['Total_Consumo_kWh'])
    
    return df

# Calcular el porcentaje de consumo por provincia respecto al total del mes
provincia_coef_mes = calcular_porcentaje_por_provincia(consumo_provincia_mes)
provincia_coef_mes.head(3)

,PROVINCIA,AÑO_MES,Consumo_kWh,%_Consumo
0,A CORUÑA,2021-10,10695536.73,1.84
1,A CORUÑA,2021-11,24993254.09,1.38
2,A CORUÑA,2021-12,31367381.77,1.37


In [28]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
provincia_coef_mes.to_csv(path + '\\Cleaning\\provincia_coef_mes.csv', index=False)